In [55]:
import pandas as pd
import re
data_df = pd.read_csv("../data/2022_sd_genomic_metadata.csv",header=0)

In [68]:
data_df = data_df[data_df['nextclade_pango'].isin(['BA.2','BA.2.12.1','BA.4','BA.5'])]
data_df['covv_collection_date'] = pd.to_datetime(data_df['covv_collection_date'])
#data_df = data_df[data_df['nextclade_pango'].isin(['BA[\.1]*.1$','BA.2','BA.2.12.1','BA.4','BA.5'])]


data_df.head(10)

,covv_virus_name,covv_accession_id,covv_collection_date,nextclade_pango,clade,zipcode
0,hCoV-19/USA/CA-SEARCH-123653/2022,EPI_ISL_14001552,2022-06-21,BA.2.12.1,22C (Omicron),92129
1,hCoV-19/USA/CA-SEARCH-123678/2022,EPI_ISL_14001569,2022-06-21,BA.5,22B (Omicron),92126
2,hCoV-19/USA/CA-SEARCH-123666/2022,EPI_ISL_14001563,2022-06-21,BA.2.12.1,22C (Omicron),92119
3,hCoV-19/USA/CA-SEARCH-123649/2022,EPI_ISL_14001549,2022-06-21,BA.5,22B (Omicron),92117
4,hCoV-19/USA/CA-SEARCH-123641/2022,EPI_ISL_14001545,2022-06-21,BA.4,22A (Omicron),92104
6,hCoV-19/USA/CA-SEARCH-123650/2022,EPI_ISL_14001550,2022-06-21,BA.2.12.1,22C (Omicron),92021
7,hCoV-19/USA/CA-SEARCH-123663/2022,EPI_ISL_14001560,2022-06-21,BA.2.12.1,22C (Omicron),92020
8,hCoV-19/USA/CA-SEARCH-123657/2022,EPI_ISL_14001555,2022-06-21,BA.5,22B (Omicron),91977
10,hCoV-19/USA/CA-SEARCH-123676/2022,EPI_ISL_14001568,2022-06-21,BA.2.12.1,22C (Omicron),91913
11,hCoV-19/USA/CA-SEARCH-123631/2022,EPI_ISL_14001538,2022-06-21,BA.2.12.1,22C (Omicron),91911


In [70]:
grouped_data = data_df.groupby(['zipcode', 'nextclade_pango'])['covv_collection_date'].min().reset_index()
grouped_data['week_appeared'] = grouped_data['covv_collection_date'].dt.isocalendar().week


In [71]:
grouped_data

,zipcode,nextclade_pango,covv_collection_date,week_appeared
0,32040,BA.2.12.1,2022-05-14,19
1,79118,BA.5,2022-06-12,23
2,83204,BA.5,2022-06-19,24
3,84770,BA.2,2022-05-17,20
4,85233,BA.5,2022-06-20,25
...,...,...,...,...
246,94547,BA.2,2022-03-28,13
247,95357,BA.2,2022-05-04,18
248,95492,BA.2.12.1,2022-05-20,20
249,95757,BA.2,2022-01-19,3


In [84]:
community_df = pd.read_csv('/data/bp-codeathon-team-2/zipcode_community.csv', index_col = False, header=0)
community_df
community_df['zipcode'] = community_df['zipcode'].astype('str')
grouped_data['zipcode'] = grouped_data['zipcode'].astype('str')

In [92]:
first_appearance_df = grouped_data.merge(community_df, how='left', on = 'zipcode')
first_appearance_df.dropna().to_csv('../data/first_appearance.csv',sep=',')